In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#import wrangle
import env
import seaborn as sns
import acquire
import prep
import scipy.stats as stats

   transactiondate   id  bathrooms  bedrooms    sqft  tax_value  \
0       2017-01-02   20        2.0       4.0  3633.0   296425.0   
1       2017-01-02   31        4.0       3.0  1620.0   847770.0   
2       2017-01-02   33        2.0       3.0  2077.0   646760.0   
3       2017-01-03  110        1.0       3.0  1244.0   169471.0   
4       2017-01-03  111        2.0       3.0  1300.0   233266.0   

  architecturalstyledesc  architecturalstyletypeid architecturalstyledesc  \
0                   None                       NaN                   None   
1                   None                       NaN                   None   
2                   None                       NaN                   None   
3                   None                       NaN                   None   
4                   None                       NaN                   None   

    id  ...  taxamount  taxdelinquencyflag  taxdelinquencyyear  \
0   20  ...    6941.39                None                 NaN   
1 

In [2]:
#single family residence from zillow DB
df = prep.prep_df()

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
#drops columns with < 80% of values filled in
df = prep.impute_values(df)
df = prep.drop_columns(df)

In [ ]:
df = df.dropna(how='any')

In [ ]:
test = df[(df.propertycountylandusecode != "010G") & ((df.propertycountylandusecode != "010M"))]

test.propertycountylandusecode.astype(float)

In [ ]:
test = df[(df.propertycountylandusecode == "010G") | ((df.propertycountylandusecode == "010M"))]

test.shape

In [ ]:
df[['landtaxvaluedollarcnt','lotsizesquarefeet']].head()

In [ ]:
df['tax_value_per_foot'] = df.landtaxvaluedollarcnt/df.lotsizesquarefeet
df.tax_value_per_foot.mean()

In [ ]:
(df.structuretaxvaluedollarcnt/df.tax_value).mean()

In [ ]:
df.head().T

In [ ]:
df.lotsizesquarefeet.head()

In [ ]:
df.landtaxvaluedollarcnt /df.tax_value_per_foot.mean()

In [ ]:
df_nulls_r = pd.DataFrame(df.apply(lambda x: df.shape[1] -x.count(),axis=1))
df_nulls_r['pct_rows_missing'] = df_nulls_r[0] / df.shape[1]
row_drops = df_nulls_r[df_nulls_r['pct_rows_missing'] >.2]
row_drops['column_names'] = row_drops.index
row_drops = list(row_drops.column_names)
#df_new = df.drop(row_drops,axis=0)

df_nulls_r.max()


In [ ]:
Q1 = df.quantile(.25)
Q3 = df.quantile(.75)
IQR = Q3-Q1

In [ ]:
IQR = Q3-Q1
Q1

In [ ]:
df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
def outlier(series):
    Q1 = series.quantile(.25)
    Q3 = series.quantile(.75)
    IQR = Q3-Q1
    
    series = series[((series < (Q1 - 3 * IQR)) |(series > (Q3 + 3 * IQR)))]
    return series

In [ ]:
def remove_outliers_iqr(df, columns):
    for col in columns:
        q75, q25 = np.percentile(df[col], [75,25])
        ub = 3*stats.iqr(df[col]) + q75
        lb = q25 - 3*stats.iqr(df[col])
        print(len(df[df[col] <= ub]))
        df = df[df[col] <= ub]
        df = df[df[col] >= lb]
    return df

In [ ]:
len(remove_outliers_iqr(df,['tax_value','sqft','lotsizesquarefeet','structuretaxvaluedollarcnt']))

In [ ]:
df.shape

In [3]:
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

import prep
import seaborn as sns
from sklearn.cluster import KMeans
 

In [4]:
train, test = train_test_split(df, train_size = .75, random_state = 123)

In [ ]:
train.head()

In [5]:
X_train = train.drop(columns=['logerror','fips','latitude','longitude','regionidcity','regionidcounty','regionidzip'])
y_train = train[['logerror']]
X_test = test.drop(columns=['logerror','fips','latitude','longitude','regionidcity','regionidcounty','regionidzip'])
y_test = test[['logerror']]
X_train.tax_value = X_train.tax_value /1000
X_train.structuretaxvaluedollarcnt = X_train.structuretaxvaluedollarcnt/1000

In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(X_train)
kmeans.predict(X_train)
#X_train['cluster'] = kmeans.predict(X_train)

In [ ]:
pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)\
    .reset_index()\
    .rename({'index': 'cluster'}, axis=1)\
    .set_index('cluster')

In [ ]:
X_train['cluster'] = kmeans.predict(X_train)
X_train.head()

In [ ]:
sns.pairplot(X_train,vars=['sqft','bedrooms','bathrooms','tax_value','lotsizesquarefeet'])

In [ ]:
X_train.describe()

In [ ]:
from sklearn.linear_model import LassoCV

reg = LassoCV()
reg.fit(X_train, y_train)

print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X_train,y_train))
coef = pd.Series(reg.coef_, index = X_train.columns)


print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
coef.sort_values()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from math import sqrt
import warnings

In [ ]:
from sklearn.linear_model import LinearRegression

lm1 = LinearRegression()

print(lm1)

In [ ]:
lm1.fit(X_train, y_train)
print("Linear Model:", lm1)

lm1_y_intercept = lm1.intercept_
print("intercept: ", lm1_y_intercept)

lm1_coefficients = lm1.coef_
print("coefficients: ", lm1_coefficients)

In [ ]:
y_pred_lm1 = lm1.predict(X_train)

In [ ]:
train.describe()

In [ ]:
    kmeans = KMeans(n_clusters=5)
    kmeans.fit(X_train['sqft','tax_value'])
    y_train['cluster'] = kmeans.predict(y_train)

In [6]:
def target_cluster(y_train,X_train):
    kmeans = KMeans(n_clusters=5)
    kmeans.fit(y_train)
    y_train['cluster'] = kmeans.predict(y_train)
    train = X_train.merge(y_train,left_index=True,right_index=True)
    return train

In [8]:
train = target_cluster(y_train,X_train)

In [17]:
train.shape

(40908, 10)

In [ ]:
y_train['cluster'] = kmeans.predict(y_train)


In [16]:
train.groupby('cluster').describe()

bathrooms                                               bedrooms  \
            count      mean       std  min  25%  50%  75%   max    count   
cluster                                                                    
0          3438.0  2.213642  0.990254  1.0  2.0  2.0  3.0  19.5   3438.0   
1          1751.0  2.295260  1.074736  1.0  2.0  2.0  3.0  10.0   1751.0   
2         35523.0  2.249106  0.980436  1.0  2.0  2.0  3.0  13.0  35523.0   
3           120.0  2.229167  0.914165  1.0  2.0  2.0  3.0   6.0    120.0   
4            76.0  2.236842  0.943305  1.0  2.0  2.0  3.0   7.0     76.0   

                   ... structuretaxvaluedollarcnt           logerror  \
             mean  ...                        75%       max    count   
cluster            ...                                                 
0        3.291158  ...                  197.95400  7340.827   3438.0   
1        3.336379  ...                  207.76550  4044.552   1751.0   
2        3.308673  ...                  204.16050  8225.708  35523.0   
3        3.341667  ...                  165.11325  1111.899    120.0   
4        3.184211  ...                  216.34975  2073.630     76.0   

                                                                               
             mean       std       min       25%       50%       75%       max  
cluster                                                                        
0       -0.177691  0.116648 -0.761094 -0.206640 -0.134660 -0.101999 -0.081219  
1        0.367658  0.187471  0.191269  0.231516  0.301110  0.438014  1.119803  
2        0.014371  0.048106 -0.081186 -0.016094  0.008502  0.036776  0.191098  
3        1.872766  0.500100  1.132791  1.368893  1.877454  2.304599  3.123634  
4       -1.365637  0.633114 -3.737018 -1.874492 -1.065844 -0.864283 -0.777944  

[5 rows x 64 columns]

In [ ]:
pd.DataFrame(.cluster_centers_, columns=train_c.columns)\
    .reset_index()\
    .rename({'index': 'cluster'}, axis=1)\
    .set_index('cluster')

In [ ]:
train_c.groupby('cluster').mean().sort_values(by='logerror')

In [ ]:
train_c.shape

In [ ]:
ax = sns.swarmplot(x='cluster',y='sqft',data=train_c)